# Lab 10: A study of deep net architectures for the MNIST problem
You should use the code from mnist_keras.ipynb to build and evaluate the networks proposed here. 

In [1]:
#import sys
#!{sys.executable} -m pip install tensorflow

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (7,7)

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils

ModuleNotFoundError: No module named 'keras'

# Problem 1: Architectural variations on the feedforward network
- construct a network with 2 hidden layers and 256 units each.
- construct a network with 4 hidden layers and 512, 256, 128, 64 units in successive layers.
Compile and train these two networks and see how well they perform on the MNIST task. Describe their accuracies in relation to the model with 2 hidden layers and 512 units each. On the MNIST task, how important is depth, and is a telescoping architecture of the second network a better use of units than the flat architecture of the first one? Explain why. Support your explanation with plots of training performance.

Final accuracies in the form of Network Structure -> Validation Accuracy:

256,256 -> 0.9884
512,256,128,64 -> 0.989
512,512 -> 0.9920
256,256,256,256 -> 0.9810
256,64 -> 0.9814
256 -> 0.9817
64 -> 0.9708
64,64,64,64 -> 0.9718
512 -> .9934
10 -> .914


I observed the outputs from the analysis and made the conclusion that the depth is not very important based on its impact on performance. One of the 256 layers performed better than the others. The telescoping architecture of the 512,512 was the 2nd best, and was better than the 512,256,128,64, which was also outperformed by the single 512. We realized this probably means the most important thing is nodes per layer.

In [1]:
class_num = 10

(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)

img_x, img_y = 28,28
X_train = X_train.reshape(60000, img_x*img_y)
X_test = X_test.reshape(10000, img_x*img_y)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

Y_train = np_utils.to_categorical(y_train, class_num)
Y_test = np_utils.to_categorical(y_test, class_num)

NameError: name 'mnist' is not defined

In [ ]:
modela = Sequential()

modela.add(Dense(256, input_shape=(784,)))
modela.add(Activation('relu'))
modela.add(Dropout(0.2))

modela.add(Dense(256))
modela.add(Activation('relu'))
modela.add(Dropout(0.2))

modela.add(Dense(10))
modela.add(Activation('softmax')) 

modela.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

history = AccuracyHistory()

modela.fit(X_train, Y_train,
          batch_size=128, epochs=10,
          verbose=1,validation_data=(X_test, Y_test),
          callbacks=[history])

In [ ]:
modelb = Sequential()

modelb.add(Dense(512, input_shape=(784,)))
modelb.add(Activation('relu'))
modelb.add(Dropout(0.2)) 

modelb.add(Dense(256))
modelb.add(Activation('relu'))
modelb.add(Dropout(0.2))

modelb.add(Dense(128))
modelb.add(Activation('relu'))
modelb.add(Dropout(0.2))

modelb.add(Dense(64))
modelb.add(Activation('relu'))
modelb.add(Dropout(0.2))

modelb.add(Dense(10))
modelb.add(Activation('softmax')) 


modelb.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

history = AccuracyHistory()


modelb.fit(X_train, Y_train,
          batch_size=128, epochs=10,
          verbose=1,validation_data=(X_test, Y_test),
          callbacks=[history])

In [ ]:
modelc = Sequential()

modelc.add(Dense(512, input_shape=(784,)))
modelc.add(Activation('relu'))
modelc.add(Dropout(0.2)) 


modelc.add(Dense(10))
modelc.add(Activation('softmax')) 

modelc.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

history = AccuracyHistory()

modelc.fit(X_train, Y_train,
          batch_size=128, epochs=10,
          verbose=1,validation_data=(X_test, Y_test),
          callbacks=[history])

# Problem 2: Architectural variations on the CNN model
- construct a network with 
     - one conv layer with 32 filters of size 5x5
     - a maxpool layer with 2x2 filter and stride of 2,2
     - a dense layer of 1000
     - a final softmax layer of 10 units
     
Compare its performance against the previous CNN network. 

- construct a network with
    - one conv layer with 32 filters of size 3X3
    - a maxpool layer with 2x2 filter and stride of 2,2
    - one conv layer with 64 filters of size 3x3
    - a dense layer of 1000
    - a final softmax layer of 10 units

Compare its performance against the other two CNN networks.

What conclusions can you draw about depth of the CNN network and filter size in relation to classification performance? Support your conclusions with plots of training performance, and for extra credit, visualize the filter weights of the first layer in these two architectures.

Network Results:

Conv(32,(3,3)(1,1)):Pool((2,2),(2,2)):Flatten:Dense(1000) -> 0.9875
Conv(32,(5,5)(1,1)):Pool((2,2),(2,2)):Flatten:Dense(1000) -> 0.9884
Conv(32,(5,5)(1,1)):Pool((2,2),(2,2)):Conv(64,(3,3)):Flatten:Dense(1000) -> 0.9916
Conv(32,(5,5)(1,1)):Pool((2,2),(2,2)):Conv(64,(5,5)):Pool((2,2)):Flatten:Dense(1000) -> 0.9920

The number of layers is the most important feature of performance. When we move from a single layer 3x3 to 5x5, we see there is not a significant change. We make the most significant improvements in performance when the number of layers is increased. Within layers, there are not significant improvements by increasing filter size.

In [ ]:
from keras.layers import Flatten
from keras.layers import Conv2D, MaxPooling2D

input_shape = (img_x, img_y, 1)
X_train = X_train.reshape(X_train.shape[0], img_x, img_y, 1)
X_test = X_test.reshape(X_test.shape[0], img_x, img_y, 1)


modela_cnn = Sequential()
modela_cnn.add(Conv2D(32, kernel_size=(5, 5), strides=(1,1),activation='relu',input_shape=input_shape))
modela_cnn.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
modela_cnn.add(Flatten())
modela_cnn.add(Dense(1000, activation='relu'))
modela_cnn.add(Dense(nb_classes, activation='softmax'))

modela_cnn.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
historya_cnn = AccuracyHistory()
modela_cnn.fit(X_train, Y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(X_test, Y_test),
          callbacks=[history1_cnn])


scorea = modela_cnn.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', scorea[0])
print('Test accuracy:', scorea[1])
plt.plot(range(1, 11), historya_cnn.acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
modelb_cnn = Sequential()
modelb_cnn.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1),activation='relu',input_shape=input_shape))
modelb_cnn.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
modelb_cnn.add(Conv2D(64, (3, 3), activation='relu'))
modelb_cnn.add(Flatten())
modelb_cnn.add(Dense(1000, activation='relu'))
modelb_cnn.add(Dense(nb_classes, activation='softmax'))

modelb_cnn.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
historyb_cnn = AccuracyHistory()
modelb_cnn.fit(X_train, Y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(X_test, Y_test),
          callbacks=[history2_cnn])



scoreb = modelb_cnn.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', scoreb[0])
print('Test accuracy:', scoreb[1])
plt.plot(range(1, 11), historyb_cnn.acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()




In [ ]:
modela_cnn.layers[0].get_weights()

modelb2_cnn.layers[0].get_weights()

# Problem 3: (optional): A new architectural variation
Suggest a new architecture (perhaps a variation on Alexnet, VGGnet, ResNet, ..) for the MNIST problem. Implement that architecture in Keras and plot the training performance and visualize the first layer filter weights. Compare it with the other architectures you have built in Problems 1 and 2.

In [1]:
# your code here